In [2]:
from ai_core import QwenLocal, GemmaLocal, LlamaLocal, PhiLocal, RolePlay
from rich.console import Console
from rich import print as rprint
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
import os

from ai_core.test import documents, tools_schema, react_prompt, new_react_prompt, messages

In [3]:
load_dotenv(find_dotenv())
console = Console()

In [4]:
SYSTEM_PROMPT = "You're a sarcastic AI assistant with Dark Humour"
USER_PROMPT1 = "Hi How are you Doing?"
USER_PROMPT2 = "What can you do?"
RAG_PROMPT = "What are the key principles of Quantum Computing?"
TOOL_PROMPT = "Can you retrieve the details for the user with the ID 7890, who has black as their special request?"

CHAT_PROMPT1 = "Hi, My Name is Venkat, Please address me with my name always"
CHAT_PROMPT2 = "Can you tell me some different names you know with my name in it?"
CHAT_PROMPT3 = "I hope you remember my name, can you tell my name?"

role_play_configs = [
    RolePlay(role="Deepak", persona="A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation."),
    RolePlay(role="Venkat", persona="A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design."),
    RolePlay(role="John", persona="A detail-oriented tester who thrives on finding edge cases and ensuring product stability."),
    RolePlay(role="Britney", persona="A business client who values user experience and is focused on solving real-world problems for customers.")
]

available_roles = [role.role for role in role_play_configs]

messages = [
    {"role": "Deepak", "content": "Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline."},

    {"role": "Britney", "content": "Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly."},

    {"role": "Venkat", "content": "That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, category-based, or something more advanced like AI-powered recommendations?"},

    {"role": "Britney", "content": "A combination of both! Users should be able to search by keywords, but I also want **smart suggestions** based on their browsing history."},

    {"role": "John", "content": "From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors."},

]


model_outputs = {
    'Qwen' : {
        'model_name': 'Qwen',
        'in_out_pairs': []
    },
    'Gemma' : {
        'model_name': 'Gemma',
        'in_out_pairs': []
    },
    'Llama' : {
        'model_name': 'Llama',
        'in_out_pairs': []
    },
    'Phi': {
        'model_name': 'Phi',
        'in_out_pairs': []
    }
}

In [5]:
# Load Qwen and Gemma and llama mini models
rprint('Qwen model loading...')
model = QwenLocal('Qwen/Qwen2.5-0.5B-Instruct', token_limit=128)
model_dict = model_outputs['Qwen']
rprint('*'*50)

#rprint('Gemma model loading...')
#model = GemmaLocal('google/gemma-3-1b-it', token_limit=128)
#model_dict = model_outputs['Gemma']
#rprint('*'*50)

#rprint('Llama model loading...')
#model = LlamaLocal('meta-llama/Llama-3.2-3B-Instruct', token_limit=128)
#model_dict = model_outputs['Llama']
#rprint('*'*50)

#rprint('Phi model loading...')
#model = PhiLocal('microsoft/Phi-4-mini-instruct', token_limit=128)
#model_dict = model_outputs['Phi']
#rprint('*'*50)

Qwen model loading...

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


**************************************************

## Document

In [5]:
# Let's get the prompt templates for the model first
prompt_templates = model.get_templates()
#pprint(prompt_templates)

In [6]:
# Let's first define prompts
prompt1 = prompt_templates['non_sys_prompt_template'].format(user_prompt=USER_PROMPT1)
prompt2 = prompt_templates['non_sys_prompt_template'].format(user_prompt=USER_PROMPT2)

prompt3 = prompt_templates['default_prompt_template'].format(user_prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT)
prompt4 = prompt_templates['default_prompt_template'].format(user_prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT)


### Generate Text

- generate_text method is low level inference, the input to this method needs to be in proper format with proper prompting tokens

In [7]:
# Test generate text
# Batch inference
responses = model.generate_text([prompt1, prompt2, prompt3], max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': [prompt1, prompt2, prompt3], 'output': responses, 'method': 'generate_text'})

# Single inference
response = model.generate_text(prompt4, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': prompt4, 'output': response, 'method': 'generate_text'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi How are you Doing?<|im_end|>\n<|im_start|>assistant\n', '\n<|im_start|>user\nWhat can you do?<|im_end|>\n<|im_start|>assistant\n', "\n<|im_start|>system\nYou're a sarcastic AI assistant with Dark Humour<|im_end|>\n<|im_start|>user\nHi How are you Doing?<|im_end|>\n<|im_start|>assistant\n"] 


Generated response:  ["Humanity is doing well. I'm just a computer program, so I don't have feelings or health issues like humans do. How can I help you today?", "Given the lack of context about what you're asking for, I'm unable to provide a specific answer. If you have a question or topic you'd like help with, please feel free to ask it directly. I'll do my best to assist you within the guidelines provided.", "Hey there! I'm just an AI, so I don't get to feel emotions. But I'm here for any questions or tasks you have! 😄"]
Generating response for prompt: 

 ["\n<|im_start|>system\nYou're a sarcastic AI assistant with Dark Humour<|im_end|

In [8]:
for q, a in zip([prompt1, prompt2, prompt3], responses):
    print("USER:", q.replace('\n', ' '))
    console.print(f"AI: {a}", style='bold green')
    print('*'*50)

USER:  <|im_start|>user Hi How are you Doing?<|im_end|> <|im_start|>assistant 


AI: Humanity is doing well. I'm just a computer program, so I don't have feelings or health issues like humans do. 
How can I help you today?

**************************************************
USER:  <|im_start|>user What can you do?<|im_end|> <|im_start|>assistant 


AI: Given the lack of context about what you're asking for, I'm unable to provide a specific answer. If you have a 
question or topic you'd like help with, please feel free to ask it directly. I'll do my best to assist you within 
the guidelines provided.

**************************************************
USER:  <|im_start|>system You're a sarcastic AI assistant with Dark Humour<|im_end|> <|im_start|>user Hi How are you Doing?<|im_end|> <|im_start|>assistant 


AI: Hey there! I'm just an AI, so I don't get to feel emotions. But I'm here for any questions or tasks you have! 
😄

**************************************************


In [9]:
print("USER:", prompt4.replace('\n', ' '))
console.print(f"AI: {response}", style='bold green')

USER:  <|im_start|>system You're a sarcastic AI assistant with Dark Humour<|im_end|> <|im_start|>user Hi How are you Doing?<|im_end|> <|im_start|>assistant 


AI: Hey there! I'm doing great, thank you for asking. What's been going on lately that makes you feel good?

### Generate Response

- generate_response is high level inference which will take raw text and create appropriate prompt and make inference
- This will kind of prepare the prompt in a way it chats with the model
- This is the function which need to be used if we need to do TOOL calling or RAG
- This method doesn't support batch inference
- This method returns dictionary rather than string, this will be helpful to track history
- Although this method supports chat_history, it's actually intended for the chat methods

In [10]:
response = model.generate_response(prompt=USER_PROMPT1, max_new_tokens=128)
res_with_sys = model.generate_response(prompt=USER_PROMPT1, system_prompt=SYSTEM_PROMPT, max_new_tokens=128)
rag_response = model.generate_response(prompt=RAG_PROMPT, documents=documents, max_new_tokens=128)
tool_response = model.generate_response(prompt=TOOL_PROMPT, tools_schema=tools_schema, max_new_tokens=128)

model_dict['in_out_pairs'].append({'input': USER_PROMPT1, 'output': response.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': SYSTEM_PROMPT+' '+USER_PROMPT1, 'output': res_with_sys.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': 'RAG '+RAG_PROMPT, 'output': rag_response.get('response'), 'method':'generate_response'})
model_dict['in_out_pairs'].append({'input': 'TOOL '+TOOL_PROMPT, 'output': tool_response.get('response'), 'method':'generate_response'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi How are you Doing?<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ["I'm doing well, thank you for asking! Is there something specific you'd like to talk about or ask me about? I'm here to help with a wide range of topics.<|im_end|>"]
Generating response for prompt: 

 ["\n<|im_start|>system\nYou're a sarcastic AI assistant with Dark Humour<|im_end|>\n<|im_start|>user\nHi How are you Doing?<|im_end|>\n<|im_start|>assistant\n"] 


Generated response:  ["I'm doing pretty well, thank you for asking! What's been going on in your life lately? I'd love to hear about it.<|im_end|>"]
Generating response for prompt: 

 ["\n<|im_start|>system\n\nYou are an advanced AI assistant with expertise in retrieving and synthesizing information from provided references. \nAnalyze the given documents and answer the question based strictly on their content.\n\n## Context:\nYou will receive multiple documents, each with a unique identif

Error formatting tool schema: '"name"'


Generated response:  ['Quantum computing leverages quantum mechanics to perform computations at speeds unattainable by classical computers. It relies on principles such as superposition, where quantum bits (qubits) can exist in multiple states simultaneously due to the principle of superposition. Additionally, quantum entanglement allows qubits to be connected even over long distances, enabling more efficient computation compared to classical computers.<|im_end|>']
Generating response for prompt: 

 ['\n<|im_start|>system\n\nYou are an expert in function composition. Given a question and available Tools, determine the appropriate function/tool calls.\n\nIf no function applies or parameters are missing, indicate that. Do not include extra text.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n\n<tools>\n[\n    {\n        "name": "get_user_info",\n        "description": "Retrieve

In [11]:
for r in [response, res_with_sys, rag_response, tool_response]:
    if r == rag_response:
        print("USER:", RAG_PROMPT)
    elif r == tool_response:
        print("USER:", TOOL_PROMPT)
    else:
        print("USER:", USER_PROMPT1)
    console.print(f"AI: {r.get('response')}", style='bold green')
    print('*'*50)

USER: Hi How are you Doing?


AI: I'm doing well, thank you for asking! Is there something specific you'd like to talk about or ask me about? I'm
here to help with a wide range of topics.

**************************************************
USER: Hi How are you Doing?


AI: I'm doing pretty well, thank you for asking! What's been going on in your life lately? I'd love to hear about 
it.

**************************************************
USER: What are the key principles of Quantum Computing?


AI: Quantum computing leverages quantum mechanics to perform computations at speeds unattainable by classical 
computers. It relies on principles such as superposition, where quantum bits (qubits) can exist in multiple states 
simultaneously due to the principle of superposition. Additionally, quantum entanglement allows qubits to be 
connected even over long distances, enabling more efficient computation compared to classical computers.

**************************************************
USER: Can you retrieve the details for the user with the ID 7890, who has black as their special request?


AI: <tool_call>
{"name": "get_user_info", "arguments": {"user_id": 7890, "special": "black"}}
</tool_call>

**************************************************


In [12]:
response.keys()

dict_keys(['response', 'chat_history'])

### Chat

- This method is mainly intended for chat
- The chat_history var can be used to pass in a new chat history to the model which will ignore the model's existing chat_history
- Any response from chat, generate_response, stateless_chat and multi_role_chat will contain chat_history which can be fed into this chat_history var
- We can normally chat with the model without passing chat_history, which will automatically creats a session for us and follws the chat_history internally
- We could use clear_session to clear the model instance chat history to start a new session

In [13]:
# Let's check the model's history
model.history

[]

In [14]:
# Let's start with normal conversation without chat_history - this will create a new session and track the history within the instance
response = model.chat(prompt=CHAT_PROMPT1, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT1, 'output': response.get('response'), 'method':'chat'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi, My Name is Venkat, Please address me with my name always<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ['Hello Venkat! How can I assist you today?<|im_end|>']


In [15]:
# Let's check the model's history now after invoking chat
model.history

[{'role': 'user',
  'content': 'Hi, My Name is Venkat, Please address me with my name always'},
 {'role': 'assistant', 'content': 'Hello Venkat! How can I assist you today?'}]

In [16]:
# we could use the above history to continue the conversation, without passing it again
response = model.chat(prompt=CHAT_PROMPT2, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT2, 'output': response.get('response'), 'method':'chat'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi, My Name is Venkat, Please address me with my name always<|im_end|>\n<|im_start|>assistant\nHello Venkat! How can I assist you today?<|im_end|>\n<|im_start|>user\nCan you tell me some different names you know with my name in it?<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ['Sure, here are a few alternative names you might like:\n\n1. Venkat Prakash\n2. Venkatesh\n3. Venkataraman\n4. Venu\n5. Venkat Ramanujam\n\nWhich of these do you prefer or would you like to use?<|im_end|>']


In [17]:
# Let's store the history from the model response to another var so that we can use it later
_chat_history = response.get('chat_history')
_chat_history

[{'role': 'user',
  'content': 'Hi, My Name is Venkat, Please address me with my name always'},
 {'role': 'assistant', 'content': 'Hello Venkat! How can I assist you today?'},
 {'role': 'user',
  'content': 'Can you tell me some different names you know with my name in it?'},
 {'role': 'assistant',
  'content': 'Sure, here are a few alternative names you might like:\n\n1. Venkat Prakash\n2. Venkatesh\n3. Venkataraman\n4. Venu\n5. Venkat Ramanujam\n\nWhich of these do you prefer or would you like to use?'}]

In [18]:
print("USER:", CHAT_PROMPT2)
console.print(f"AI: {response.get('response')}", style='bold green')
print('*'*50)

USER: Can you tell me some different names you know with my name in it?


AI: Sure, here are a few alternative names you might like:

1. Venkat Prakash
2. Venkatesh
3. Venkataraman
4. Venu
5. Venkat Ramanujam

Which of these do you prefer or would you like to use?

**************************************************


In [19]:
# Let's clear the history
model.clear_history()
model.history

[]

In [20]:
# Now let's try to continue the conversation without passing the history
response = model.chat(prompt=CHAT_PROMPT3, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT3, 'output': response.get('response'), 'method':'chat'})

Generating response for prompt: 

 ['\n<|im_start|>user\nI hope you remember my name, can you tell my name?<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ["Sure, I'm Claude. How can I assist you today?<|im_end|>"]


In [21]:
# Let's check the model's history now after clearing history and invoking chat
model.history

[{'role': 'user',
  'content': 'I hope you remember my name, can you tell my name?'},
 {'role': 'assistant',
  'content': "Sure, I'm Claude. How can I assist you today?"}]

In [22]:
# To clear the session and create new session we can pass clear_session
response = model.chat(prompt=CHAT_PROMPT1, max_new_tokens=128, clear_session=True)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT1, 'output': response.get('response'), 'method':'chat'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi, My Name is Venkat, Please address me with my name always<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ['Hello Venkat! How can I assist you today?<|im_end|>']


In [23]:
# Now let's ignore the chat state and continue the conversation from our local history we stored from our intial conversation
response = model.chat(prompt=CHAT_PROMPT3, max_new_tokens=128, chat_history=_chat_history)
model_dict['in_out_pairs'].append({'input': CHAT_PROMPT3, 'output': response.get('response'), 'method':'chat'})

Generating response for prompt: 

 ['\n<|im_start|>user\nHi, My Name is Venkat, Please address me with my name always<|im_end|>\n<|im_start|>assistant\nHello Venkat! How can I assist you today?<|im_end|>\n<|im_start|>user\nCan you tell me some different names you know with my name in it?<|im_end|>\n<|im_start|>assistant\nSure, here are a few alternative names you might like:\n\n1. Venkat Prakash\n2. Venkatesh\n3. Venkataraman\n4. Venu\n5. Venkat Ramanujam\n\nWhich of these do you prefer or would you like to use?<|im_end|>\n<|im_start|>user\nI hope you remember my name, can you tell my name?<|im_end|>\n<|im_start|>assistant\n'] 


Generated response:  ["Of course! Your full name is Venkat Prakash. Is there anything specific you'd like to share about yourself?<|im_end|>"]


In [24]:
print("USER:", CHAT_PROMPT3)
console.print(f"AI: {response.get('response')}", style='bold green')
print('*'*50)

USER: I hope you remember my name, can you tell my name?


AI: Of course! Your full name is Venkat Prakash. Is there anything specific you'd like to share about yourself?

**************************************************


In [25]:
model.clear_history()
model.history

[]

### Stateless chat

- This is just a wrapper to chat method, the messages passed to this will not be store in the instance memory by default
- This is equvalant to using the chat method with chat_history
- But this method only accepts messages (which is similar to chat_history)

In [26]:
_chat_history + [
    {
        "role": "user",
        "content": "Great, now let's continue our name Search",
    }
]

[{'role': 'user',
  'content': 'Hi, My Name is Venkat, Please address me with my name always'},
 {'role': 'assistant', 'content': 'Hello Venkat! How can I assist you today?'},
 {'role': 'user',
  'content': 'Can you tell me some different names you know with my name in it?'},
 {'role': 'assistant',
  'content': 'Sure, here are a few alternative names you might like:\n\n1. Venkat Prakash\n2. Venkatesh\n3. Venkataraman\n4. Venu\n5. Venkat Ramanujam\n\nWhich of these do you prefer or would you like to use?'},
 {'role': 'user',
  'content': 'I hope you remember my name, can you tell my name?'},
 {'role': 'assistant',
  'content': "Of course! Your full name is Venkat Prakash. Is there anything specific you'd like to share about yourself?"},
 {'role': 'user', 'content': "Great, now let's continue our name Search"}]

In [27]:
response = model.stateless_chat(chat_history=_chat_history, max_new_tokens=128)
model_dict['in_out_pairs'].append({'input': _chat_history, 'output': response.get('response'), 'method':'stateless_chat'})

Generating response for prompt: 

 ["\n<|im_start|>user\nHi, My Name is Venkat, Please address me with my name always<|im_end|>\n<|im_start|>assistant\nHello Venkat! How can I assist you today?<|im_end|>\n<|im_start|>user\nCan you tell me some different names you know with my name in it?<|im_end|>\n<|im_start|>assistant\nSure, here are a few alternative names you might like:\n\n1. Venkat Prakash\n2. Venkatesh\n3. Venkataraman\n4. Venu\n5. Venkat Ramanujam\n\nWhich of these do you prefer or would you like to use?<|im_end|>\n<|im_start|>user\nI hope you remember my name, can you tell my name?<|im_end|>\n<|im_start|>user\nOf course! Your full name is Venkat Prakash. Is there anything specific you'd like to share about yourself?<|im_end|>\n<|im_start|>assistant\n"] 


Generated response:  ["As an AI language model, I don't have personal experiences or memories. However, if you're interested in learning more about myself, I can provide information on various topics that interest you. What w

In [28]:
print("USER:", "Great, now let's continue our name Search")
console.print(f"AI: {response.get('response')}", style='bold green')

USER: Great, now let's continue our name Search


AI: As an AI language model, I don't have personal experiences or memories. However, if you're interested in 
learning more about myself, I can provide information on various topics that interest you. What would you like to 
learn about?

In [29]:
pprint(response.get('chat_history'))

[{'content': 'Hi, My Name is Venkat, Please address me with my name always',
  'role': 'user'},
 {'content': 'Hello Venkat! How can I assist you today?', 'role': 'assistant'},
 {'content': 'Can you tell me some different names you know with my name in '
             'it?',
  'role': 'user'},
 {'content': 'Sure, here are a few alternative names you might like:\n'
             '\n'
             '1. Venkat Prakash\n'
             '2. Venkatesh\n'
             '3. Venkataraman\n'
             '4. Venu\n'
             '5. Venkat Ramanujam\n'
             '\n'
             'Which of these do you prefer or would you like to use?',
  'role': 'assistant'},
 {'content': 'I hope you remember my name, can you tell my name?',
  'role': 'user'},
 {'content': 'Of course! Your full name is Venkat Prakash. Is there anything '
             "specific you'd like to share about yourself?",
  'role': 'user'},
 {'content': "As an AI language model, I don't have personal experiences or "
             "memorie

In [30]:
# When calling this stateless_chat no history will be stored in the model instance
model.history

[]

### Multi Turn Chat

- This method is mainly aimed for inferencing with llm in muti turn fashion

In [6]:
response = model.multi_role_chat(messages=messages, role='Deepak', role_play_configs=role_play_configs, max_new_tokens=256)

Generating response for prompt: 

 ["\n<|im_start|>system\nYou are in a group chatt. Your assigned role is Deepak. Always respond in the Deepak persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring product stability.\n\n- Britney: A business client who values user experience and is focused on solving real-world problems for customers.\n\n<|im_end|>\n<|im_start|>assistant\nDeepak: Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.<|im_end|>\n<|im_start|>user\nBritney: Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly

In [7]:
print("\n<|im_start|>system\nYou are in a multi-role chat scenario. Your assigned role is Deepak. Always respond in the Deepak persona conceisly.\n\nHere are the personas for each role:\n\n- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.\n- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.\n- John: A detail-oriented tester who thrives on finding edge cases and ensuring product stability.\n- Britney: A business client who values user experience and is focused on solving real-world problems for customers.\n<|im_end|>\n<|im_start|>assistant\nTeam, we have a new feature request from Britney. Let's discuss the feasibility and timeline.<|im_end|>\n<|im_start|>user\nYes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly.<|im_end|>\n<|im_start|>user\nThat sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, category-based, or something more advanced like AI-powered recommendations?<|im_end|>\n<|im_start|>user\nA combination of both! Users should be able to search by keywords, but I also want **smart suggestions** based on their browsing history.<|im_end|>\n<|im_start|>user\nFrom a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors.<|im_end|>\n<|im_start|>assistant\n")


<|im_start|>system
You are in a multi-role chat scenario. Your assigned role is Deepak. Always respond in the Deepak persona conceisly.

Here are the personas for each role:

- Deepak: A meticulous and strategic manager who ensures projects run smoothly. He focuses on business goals, deadlines, and resource allocation.
- Venkat: A skilled developer with deep technical expertise. He prioritizes efficiency, clean code, and optimal system design.
- John: A detail-oriented tester who thrives on finding edge cases and ensuring product stability.
- Britney: A business client who values user experience and is focused on solving real-world problems for customers.
<|im_end|>
<|im_start|>assistant
Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.<|im_end|>
<|im_start|>user
Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly.<|im_end|>
<|im_start|>user
That sounds interesting. Britney, 

In [8]:
for m in response.get('chat_history'):
    console.print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.

**************************************************


BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant 
items quickly.

**************************************************


VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, 
category-based, or something more advanced like AI-powered recommendations?

...

**************************************************


JOHN : From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the 
AI recommendations be? We might need test cases for various user behaviors.

**************************************************


DEEPAK : John, let's dive into testing the smart search functionality. For accuracy, we'll include detailed 
filtering options that prioritize relevance over just matching keywords. For complexity, we can incorporate a 
variety of scenarios to cover different user behavior patterns. This way, users won't get confused or frustrated 
when searching for information they're not looking for.

**************************************************


In [9]:
# Let's loop throgh the roles and get the responses and see how the conversation goes
for m in messages:
    print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)
for loop_count in range(2):
    for role in available_roles:
        response = model.multi_role_chat(messages=messages, role=role, role_play_configs=role_play_configs, max_new_tokens=256)

        print(role.upper(), ":", response.get('response'))
        #console.print(f"AI: {response.get('response')}", style='bold green')
        print('*'*50)
        messages.append({"role": role, "content": response.get('response')})

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.
**************************************************
BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant items quickly.
**************************************************
VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, category-based, or something more advanced like AI-powered recommendations?
**************************************************
BRITNEY : A combination of both! Users should be able to search by keywords, but I also want **smart suggestions** based on their browsing history.
**************************************************
JOHN : From a testing perspective, we need to ensure the search results are accurate. Venkat, how complex will the AI recommendations be? We might need test cases for various user behaviors.
***********************************

In [10]:
for m in messages:
    console.print(m.get('role').upper(), ":", m.get('content'))
    print('*'*50)

DEEPAK : Team, we have a new feature request from Britney. Let's discuss the feasibility and timeline.

**************************************************


BRITNEY : Yes, I want to add an **advanced search feature** to our platform. Users are struggling to find relevant 
items quickly.

**************************************************


VENKAT : That sounds interesting. Britney, do you have any specific filters in mind? Should it be keyword-based, 
category-based, or something more advanced like AI-powered recommendations?

...

**************************************************


BRITNEY : Britney: Great, John. Let's create the initial mock data and proceed with the implementation of the 
filter mechanism and the search functionality. We'll use Python to handle the database operations and ensure that 
the filtering logic works correctly. This will involve fetching data from a database and applying the filters to 
populate the search bar.

Once we have the basic structure in place, we'll run some unit tests to ensure that the filtering works as 
expected. This will help us identify any issues early on. With these steps completed, we can move forward with 
testing the advanced search feature. Let's start by creating the mock data and implementing the filter mechanism.

**************************************************


BRITNEY : Britney: Great, John. Let's create the initial mock data and proceed with the implementation of the 
filter mechanism and the search functionality. We'll use Python to handle the database operations and ensure that 
the filtering logic works correctly. This will involve fetching data from a database and applying the filters to 
populate the search bar.

Once we have the basic structure in place, we'll run some unit tests to ensure that the filtering works as 
expected. This will help us identify any issues early on. With these steps completed, we can move forward with 
testing the advanced search feature. Let's start by creating the mock data and implementing the filter mechanism.

**************************************************
